In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os

# Load environment variables from the .env file
load_dotenv()

# Get the API key from the environment variable

open_api_key = os.getenv('OPENAI_API_KEY')

# Initialize OpenAI client

openai_client = OpenAI(api_key = open_api_key)



In [ ]:
# import os
# from PyPDF2 import PdfReader, PdfWriter


# from modules.pdf_extraction import select_pdf_file
# from modules.pdf_extraction import extract_text_from_pages

# pdf_path = select_pdf_file()

# def split_pdf(input_path, output_path_1, output_path_2, split_page):
#     """
#     Split a PDF file into two separate PDF files.
    
#     Args:
#         input_path (str): Path to the input PDF file
#         output_path_1 (str): Path where to save the first part
#         output_path_2 (str): Path where to save the second part
#         split_page (int): The page number where to split (this page will be the first page of the second PDF)
#     """
#     try:
#         # Create PDF reader object
#         reader = PdfReader(input_path)
        
#         # Get total number of pages
#         total_pages = len(reader.pages)
        
#         if split_page >= total_pages:
#             raise ValueError("Split page number cannot be greater than total pages")
        
#         # Create two PDF writer objects
#         writer1 = PdfWriter()
#         writer2 = PdfWriter()
        
#         # Add pages to first output PDF (before split point)
#         for page in range(split_page):
#             writer1.add_page(reader.pages[page])
            
#         # Add pages to second output PDF (from split point to end)
#         for page in range(split_page, total_pages):
#             writer2.add_page(reader.pages[page])
            
#         # Save the first part
#         with open(output_path_1, 'wb') as output1:
#             writer1.write(output1)
            
#         # Save the second part
#         with open(output_path_2, 'wb') as output2:
#             writer2.write(output2)
            
#         return True
    
#     except Exception as e:
#         print(f"An error occurred: {str(e)}")
#         return False

# def main():
#     # Example usage
#     input_file = pdf_path  # Replace with your PDF file
#     output_file1 = "part1.pdf"
#     output_file2 = "part2.pdf"
#     split_at_page = 1000   # Split after first page (0-based index)
    
#     if os.path.exists(input_file):
#         success = split_pdf(input_file, output_file1, output_file2, split_at_page)
#         if success:
#             print(f"PDF split successfully!")
#             print(f"First part saved as: {output_file1}")
#             print(f"Second part saved as: {output_file2}")
#         else:
#             print("Failed to split PDF.")
#     else:
#         print(f"Input file '{input_file}' not found.")
        
# main()


In [ ]:
# 15

from modules.pdf_extraction import select_pdf_file
from modules.pdf_extraction import extract_text_from_pages

pdf_path = select_pdf_file()
page_no = 0
extracted_text = extract_text_from_pages(pdf_path, pages=page_no)


In [ ]:
import pymupdf 
## 111
import base64
import requests

import importlib
import modules.llm
import modules.pdf_extraction


from modules.pdf_extraction import get_page_pixel_data



# this may have to happen on a page by page basis

base64_image = get_page_pixel_data(pdf_path=pdf_path, page_no=page_no, 
                    dpi = 500, image_type = 'png')

In [1]:
from modules.pdf_extraction import select_pdf_file

from modules.pdf_extraction import get_validated_table_info
from modules.pdf_extraction import get_page_pixel_data
from modules.pdf_extraction import process_tables_to_df
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
import asyncio
import os
import pymupdf
import time


file_name = 'test_7'    
user_text='Extract all data from the table(s).'

# 1. Load Credientials

# Load environment variables from the .env file
load_dotenv()
# Get the API key from the environment variable
open_api_key = os.getenv('OPENAI_API_KEY')
# Initialize OpenAI client
openai_client = OpenAI(api_key = open_api_key)





# 2. Select PDF file and extract text
pdf_path = select_pdf_file()
doc = pymupdf.open(pdf_path)
total_pages = doc.page_count  # total number of pages in the document
page_indices = range(total_pages) # page_indices = range(1562,1567)

# page_indices can be a list of page numbers to process


# Start timing
start_time = time.time()

async def process_page():
    tasks = []
    results_output = []
    # Create all tasks first 
    async with asyncio.TaskGroup() as tg:
        for page_no in page_indices:
            page = doc.load_page(page_no)
            extracted_text = page.get_text()
            
            # extracted_text = extract_text_from_pages(pdf_path, pages=page_no)
            base64_image = get_page_pixel_data(pdf_path=pdf_path, page_no=page_no, 
                                dpi = 500, image_type = 'png')
        
            num_tables, table_headers, table_location, confidence_score_0 = await get_validated_table_info(
                text_input=extracted_text, 
                open_api_key=open_api_key, 
                base64_image=base64_image
            )

            if num_tables == 0:
                print(f"No tables found on page {page_no + 1}, skipping...")
                continue
    
            tasks.append(tg.create_task(process_tables_to_df(
                table_headers, 
                table_location,
                user_text, 
                extracted_text, 
                base64_image, 
                open_api_key,
                page_number=page_no)))
            
        # Await all tasks to complete
        for task in tasks:
            results_output.append(await task)
    
    if not results_output:
        raise ValueError("No tables found on any of the processed pages")
            
    # df_out_1 = pd.concat(results_output, ignore_index=True)
    return results_output


output_final = await process_page()


2025-03-01 14:02:54,329 - INFO - Opening file selection dialog.
2025-03-01 14:02:56,400 - INFO - Selected PDF file: C:/Users/derri/Desktop/extracted 1 and 2.pdf
2025-03-01 14:02:56,443 - INFO - Converting PDF page 1 to base64 image. DPI=500, Format=png
2025-03-01 14:02:58,751 - INFO - Finished converting page to base64.
2025-03-01 14:02:58,756 - INFO - Validating table information with multiple LLM calls.
2025-03-01 14:03:08,368 - INFO - Initial table info match or same table count. Returning first attempt's result.
2025-03-01 14:03:08,376 - INFO - Converting PDF page 2 to base64 image. DPI=500, Format=png
2025-03-01 14:03:10,512 - INFO - Finished converting page to base64.
2025-03-01 14:03:10,519 - INFO - Validating table information with multiple LLM calls.
2025-03-01 14:03:10,520 - INFO - Processing tables to DataFrame for page 1
2025-03-01 14:03:20,940 - INFO - Initial table info match or same table count. Returning first attempt's result.
2025-03-01 14:03:20,940 - INFO - Processin

In [ ]:
from modules.pdf_extraction import write_output_final
write_output_final(output_final, excel_path='files/Tester_1.xlsx', option=1, gap_rows=2)

In [ ]:
# import pymupdf

# # Open some document, for example a PDF (could also be EPUB, XPS, etc.)
# doc = pymupdf.open(pdf_path)

# # Load a desired page. This works via 0-based numbers
# page = doc[0]  # this is the first page

# # Look for tables on this page and display the table count
# tabs = page.find_tables()
# num_tables = len(tabs.tables)